Zacznijmy od wczytania niezbędnych pakietów:

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import os
from matplotlib import pyplot as plt

tf.__version__ 

# MLP dla zadania regresji

W Keras możemy tworzyć modele na dwa różne sposoby:
- zbudować model sekwencyjny - nakładamy nowe warstwy na poprzednie. Nie możemy używać wielu inputów i outputów w modelu.
- używając API funkcyjnego - pozwala na uzycie wieli inputów i uotputów.

Zaczniemy od modelu sekwencyjnego. Musimy zacząć od inicjalizacji modelu:

In [ ]:
boston_train_X = pd.read_csv("data/boston_train_X", sep=" ").to_numpy()
boston_test_X = pd.read_csv("data/boston_test_X", sep=" ").to_numpy()
boston_train_Y = pd.read_csv("data/boston_train_Y", sep=" ").to_numpy()
boston_test_Y = pd.read_csv("data/boston_test_Y", sep=" ").to_numpy()

print(boston_train_X.shape)
print(boston_train_Y.shape)

In [ ]:
boston_train_X

In [ ]:
boston_train_Y

In [ ]:
boston_model = tf.keras.Sequential()

W następnym kroku możemy dodać kilka warstw (pamiętaj, że nie musimy ponownie przypisywać modelu za pomocą `=`):

In [ ]:
boston_model.add(
    tf.keras.layers.Dense(units = 16, # Liczba neuronów w warstwie ukrytej
                             activation = "tanh", # Funkcja aktywacji
                             input_shape = (13,)) # Liczba predyktorów - tylko w pierwszej warstwie
)
boston_model.summary()

Dlaczego mamy 224 parametry?

In [ ]:
13 * 16 + 16

Po dodaniu warstwy ukrytej możemy dodać warstwę wyjściową:

In [ ]:
boston_model.add(
    tf.keras.layers.Dense(units = 1, # Liczba neuronów w warstwie wyjścia - chcemy dostać 1 liczbę
                             activation = "linear") # Funkcja liniowa dla regresji
)
boston_model.summary()

Możemy teraz skonfigurować model do treningu. Użyjemy SGD jako optymalizatora, MSE jako funkcji straty i dodamy MAE jako dodatkową metrykę.

In [ ]:
boston_model.compile(
    optimizer = "sgd",
    loss = "mse",
    metrics = ("mae"))

Jesteśmy gotowi do trenowania naszej pierwszej sieci neuronowej:

In [ ]:
history = boston_model.fit(
    x = boston_train_X,
    y = boston_train_Y,
    validation_split = 0.2, # 20% zbioru uczącego przeznaczonego na walidację
    epochs = 5, # Liczba "pętli/przejść" bo całym zbiorze treningowym
    batch_size = 30, # Wielkość próbki (batcha) dla jednej iteracji algorytmu SGD
    verbose = 1)

In [ ]:
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('model MAE')
plt.ylabel('MAE')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

Możemy teraz ocenić wytrenowany model w testowym zestawie danych:

In [ ]:
boston_model.evaluate(boston_test_X, boston_test_Y)

I policzyć predykcje:

In [ ]:
boston_predictions = boston_model.predict(boston_test_X)
boston_predictions

Na koniec możemy zapisać nasz model na dysku twardym:

In [ ]:
if not os.path.exists("models"):
    os.mkdir("models")
boston_model.save("models/boston_model.hdf5")

# MLP dla zadania klasyfikacji binarnej

Budując MLP (lub inną sieć neuronową) dla zadania klasyfikacji musimy zmienić tylko kilka drobnych detali w naszym kodzie. Zacznijmy od wczytania zbioru danych z informacjami o fraudach na kartach kredytowych i inicjalizacji modelu:

In [ ]:
creditcard_train_X = pd.read_csv("data/creditcard_train_X", sep=" ").to_numpy()
creditcard_test_X = pd.read_csv("data/creditcard_test_X", sep=" ").to_numpy()
creditcard_train_Y = pd.read_csv("data/creditcard_train_Y", sep=" ").to_numpy()
creditcard_test_Y = pd.read_csv("data/creditcard_test_Y", sep=" ").to_numpy()

print(creditcard_train_X.shape)
print(creditcard_train_Y.shape)

Zanim zbudujemy model, przekształcimy wektor odpowiedzi (klas) do formatu one-hot-encoding:

In [ ]:
creditcard_train_Y = tf.keras.utils.to_categorical(creditcard_train_Y, 2)
creditcard_test_Y = tf.keras.utils.to_categorical(creditcard_test_Y, 2)
print(creditcard_train_Y.shape)

In [ ]:
creditcard_train_Y

In [ ]:
creditcard_model = tf.keras.Sequential()

Następnie dodajmy warstwy ukryte:

In [ ]:
creditcard_model.add(
    tf.keras.layers.Dense(units = 20, # Liczba neuronów w warstwie ukrytej
                             activation = "relu", # Funkcja aktywacji
                             input_shape = (29,)) # Liczba predyktorów - tylko w pierwszej warstwie
)
creditcard_model.add(
    tf.keras.layers.Dense(units = 10, # Liczba neuronów w warstwie ukrytej
                             activation = "relu") # Funkcja aktywacji
)
creditcard_model.add(
    tf.keras.layers.Dense(units = 5, # Liczba neuronów w warstwie ukrytej
                             activation = "relu") # Funkcja aktywacji
)
creditcard_model.summary()

Oraz warstwę wyjściową:

In [ ]:
creditcard_model.add(
    tf.keras.layers.Dense(units = 2, # Liczba neuronów w warstwie wyjścia - dwie klasy
                             activation = "softmax") # Funkcja aktywacji - softmax dla klasykikacji
)
creditcard_model.summary()

Możemy teraz skonfigurować model do treningu. Użyjemy SGD jako optymalizatora, Entropii Krzyżowej jako funkcji straty i dodamy Accuracy jako dodatkową metrykę.

In [ ]:
creditcard_model.compile(
    optimizer = "sgd",
    loss = "categorical_crossentropy",
    metrics = ("accuracy"))

I wytrenować go:

In [ ]:
history = creditcard_model.fit(
    x = creditcard_train_X,
    y = creditcard_train_Y,
    validation_split = 0.2, # 20% zbioru uczącego przeznaczonego na walidację
    epochs = 5, # Liczba "pętli/przejść" bo całym zbiorze treningowym
    batch_size = 256, # Wielkość próbki (batcha) dla jednej iteracji algorytmu SGD
    verbose = 1)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
creditcard_model.evaluate(creditcard_test_X, creditcard_test_Y)

In [ ]:
creditcard_predictions = creditcard_model.predict(creditcard_test_X)
creditcard_predictions

Model może wydawać się dobry, jednakże wcale taki nie jest:

In [ ]:
pd.crosstab(creditcard_test_Y[:, 1], np.argmax(creditcard_model.predict(creditcard_test_X), axis=-1),
           rownames = ["true"], colnames = ["predicted"])

# Dropout i checkpointy

Jednym z powodów przez które nasz model zawiódł może być brak regularyzacji, która jest niezwykle ważna w przypadku mocno niezbalansowanych zbiorów lub kiepska architektura sieci:

In [ ]:
creditcard_model2 = tf.keras.Sequential()
creditcard_model2.add(
    tf.keras.layers.Dense(units = 256, # Liczba neuronów w warstwie ukrytej
                             activation = "relu", # Funkcja aktywacji
                             input_shape = (29,)) # Liczba predyktorów - tylko w pierwszej warstwie
)
creditcard_model2.add(tf.keras.layers.Dropout(0.1))
creditcard_model2.add(
    tf.keras.layers.Dense(units = 256, # Liczba neuronów w warstwie ukrytej
                             activation = "relu") # Funkcja aktywacji
)
creditcard_model2.add(tf.keras.layers.Dropout(0.1))
creditcard_model2.add(
    tf.keras.layers.Dense(units = 2, # Liczba neuronów w warstwie wyjścia
                             activation = "softmax") # Funkcja aktywacji
)
creditcard_model2.summary()

Trenowanie sieci neuronowej może zająć dużo czasu, a rozwiązanie rzeczywistego problemu może zająć dni, tygodnie, a nawet miesiące. W tym czasie wiele rzeczy może pójść nie tak, na przykład jeśli twój komputer zresetuje się z nieznanego powodu, stracisz cały postęp i dużo czasu! Aby rozwiązać ten problem, możemy dodać punkt kontrolny modelu, który będzie zapisywać model w każdej epoce. Punkt kontrolny modelu jest jednym z wielu wywołań zwrotnych, których możesz używać w Keras podczas procesu szkolenia.

Kolejnym przydatnym callbackiem jest **Early stopping** umożliwiający zatrzymanie procesu uczenia jeśli wyniki sieci nie ulegają polepszeniu. Jest to kolejna metoda regularyzacji zapobiegająca przeuczeniu modelu.

In [ ]:
creditcard_model2.compile(
    optimizer = "sgd",
    loss = "categorical_crossentropy",
    metrics = ("accuracy"))

In [ ]:
history = creditcard_model2.fit(
    x = creditcard_train_X,
    y = creditcard_train_Y,
    validation_split = 0.2, # 20% zbioru uczącego przeznaczonego na walidację
    epochs = 100, # Liczba "pętli/przejść" bo całym zbiorze treningowym
    batch_size = 256, # Wielkość próbki (batcha) dla jednej iteracji algorytmu SGD
    verbose = 1,
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 10),
                tf.keras.callbacks.ModelCheckpoint(filepath = "models/creditcard_best.hdf5",
                                                  monitor = "val_loss", save_best_only = True)]
)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
creditcard_model2.evaluate(creditcard_test_X, creditcard_test_Y)

In [ ]:
pd.crosstab(creditcard_test_Y[:, 1], np.argmax(creditcard_model2.predict(creditcard_test_X), axis=-1),
           rownames = ["true"], colnames = ["predicted"])

# MLP dla zadania regresji wieloklasowej - praca domowa

W podobny sposób możemy zbudować model sekwencyjny dla problemu klasyfikacji wieloklasowej. Korzystając ze zbioru FASHION MNIST zbuduj sieć MLP, która klasyfikuje dany obraz do jedne z 10 klas:

 - 0	T-shirt/top
 - 1	Trouser
 - 2	Pullover
 - 3	Dress
 - 4	Coat
 - 5	Sandal
 - 6	Shirt
 - 7	Sneaker
 - 8	Bag
 - 9	Ankle boot

In [ ]:
fashion_mnist_train_X = pd.read_csv("data/fashion_mnist_train_X", sep=" ").to_numpy()
fashion_mnist_test_X = pd.read_csv("data/fashion_mnist_test_X", sep=" ").to_numpy()
fashion_mnist_train_Y = pd.read_csv("data/fashion_mnist_train_Y", sep=" ").to_numpy()
fashion_mnist_test_Y = pd.read_csv("data/fashion_mnist_test_Y", sep=" ").to_numpy()

fashion_mnist_train_X = fashion_mnist_train_X / 255
fashion_mnist_test_X = fashion_mnist_test_X / 255

print(fashion_mnist_train_X.shape)
print(fashion_mnist_train_Y.shape)

In [ ]:
import matplotlib.pyplot as plt
plt.matshow(fashion_mnist_train_X[0,:].reshape((28,28)))

In [ ]:
# TASK: Create MLP for fashion MNIST classification.
# Change labels vectors to one-hot-encoding matrix using to_categorical() method

# Model architecture:
# Dense layer with 512 units and "relu" activation
# Dropout layer with 20% drop rate
# Dense layer with 512 units and "relu" activation
# Dropout layer with 20% drop rate
# Output dense layer (how many units and what activation should You use?)

# Set SGD as optimizer and use categorical crossentropy as loss function. Use accuracy as additional metric.

# Fit the model. Use 20% of the data for validation, 20 epochs and 128 samples for batch size.
# Use model checkpoint and early stopping.

# Evaluate model on test set.